In [2]:
import pandas as pd
import random
import string
from datetime import datetime

In [34]:
resultados_top_ctr = pd.read_csv('/home/pepino/Desktop/Maestría Udesa/Segundo Año/Programación Avanzada para grandes volúmenes de datos/TP/resultados_top_ctr.csv', header = 0)

In [35]:
resultados_top_ctr.head()

,log,advertiser_id,product_id
0,2023-04-23,03KNVBO915KY2ZPGA57J,ffkh8f
1,2023-04-23,03KNVBO915KY2ZPGA57J,0zmyxz
2,2023-04-23,03KNVBO915KY2ZPGA57J,28ycmv
3,2023-04-23,03KNVBO915KY2ZPGA57J,2b9w5m
4,2023-04-23,03KNVBO915KY2ZPGA57J,3calwd


In [3]:
datetime.today().strftime("%Y-%m-%d")

'2023-05-06'

In [37]:
resultados_top_ctr = pd.read_csv('/home/pepino/Desktop/Maestría Udesa/Segundo Año/Programación Avanzada para grandes volúmenes de datos/TP/resultados_top_ctr.csv', header = 0)
resultados_top_product = pd.read_csv('/home/pepino/Desktop/Maestría Udesa/Segundo Año/Programación Avanzada para grandes volúmenes de datos/TP/resultados_top_product.csv', header = 0)

In [38]:
union = resultados_top_ctr.merge(resultados_top_product, 
                                 on = ['advertiser_id','log'], 
                                 how= 'outer')

In [39]:
resultados_top_ctr.set_index(['log','advertiser_id']).head()

product_id
log        advertiser_id                  
2023-04-23 03KNVBO915KY2ZPGA57J     ffkh8f
           03KNVBO915KY2ZPGA57J     0zmyxz
           03KNVBO915KY2ZPGA57J     28ycmv
           03KNVBO915KY2ZPGA57J     2b9w5m
           03KNVBO915KY2ZPGA57J     3calwd

In [40]:
#resultados_top_ctr.set_index(['log','advertiser_id'])[(resultados_top_ctr.log == '2023-04-23') & (resultados_top_ctr.advertiser_id == '03KNVBO915KY2ZPGA57J')]

In [41]:
resultados_top_ctr.groupby('advertiser_id').head(7).head(10)

,log,advertiser_id,product_id
0,2023-04-23,03KNVBO915KY2ZPGA57J,ffkh8f
1,2023-04-23,03KNVBO915KY2ZPGA57J,0zmyxz
2,2023-04-23,03KNVBO915KY2ZPGA57J,28ycmv
3,2023-04-23,03KNVBO915KY2ZPGA57J,2b9w5m
4,2023-04-23,03KNVBO915KY2ZPGA57J,3calwd
5,2023-04-23,03KNVBO915KY2ZPGA57J,4bu13h
6,2023-04-23,03KNVBO915KY2ZPGA57J,4hzw8j
20,2023-04-23,1OW4LWJSBJK6E1D9FDH7,a8hf9j
21,2023-04-23,1OW4LWJSBJK6E1D9FDH7,023w1n
22,2023-04-23,1OW4LWJSBJK6E1D9FDH7,0xfk8v


In [42]:
resultados_top_ctr.log.min()

'2023-04-23'

# Creamos una base completa con historia

In [3]:
import datetime
from datetime import timedelta
import numpy as np

In [4]:
# Levantamos la data
usuarios = pd.read_csv('/home/pepino/Desktop/Maestría Udesa/Segundo Año/Programación Avanzada para grandes volúmenes de datos/TP/advertiser_ids')
total_logs = pd.read_csv('/home/pepino/Desktop/Maestría Udesa/Segundo Año/Programación Avanzada para grandes volúmenes de datos/TP/ads_views')
total_products = pd.read_csv('/home/pepino/Desktop/Maestría Udesa/Segundo Año/Programación Avanzada para grandes volúmenes de datos/TP/product_views.csv')

# Usuarios inactivos
usuarios_inactivos = usuarios.tail(5)['advertiser_id'].to_list()

In [5]:
print(total_logs.date.min())
print(total_logs.date.max())

# Días únicos
dias_unicos = list(total_logs.date.unique())
print("Cantidad de días únicos: ", len(dias_unicos))

2023-04-01
2023-05-30
Cantidad de días únicos:  60


In [6]:
%%time

resultados_totales_top_ctr = pd.DataFrame(columns = ['log', 'advertiser_id', 'product_id'])
resultados_totales_top_product = pd.DataFrame(columns = ['log', 'advertiser_id', 'product_id'])

for i in range(1,60):

    # Vamos a simular la base que resulte de aplicar todos los días
    yesterday = dias_unicos[i-1]

    # Nos quedamos con la base filtrada
    datos_top_ctr = total_logs[(~total_logs['advertiser_id'].isin(usuarios_inactivos)) & (total_logs['date'] == yesterday)]
    datos_top_pro = total_products[(~total_products['advertiser_id'].isin(usuarios_inactivos)) & (total_products['date'] == yesterday)]

    ########################
    # Calculamos el TOPCTR #
    ########################

    # Hacemos las agrupaciones
    top_ctr = datos_top_ctr.groupby(['advertiser_id','product_id'])['type'].value_counts().reset_index().pivot(index = ['advertiser_id','product_id'], columns='type', values='count').reset_index()
    top_ctr.fillna(0, inplace = True)

    # Hacemos el ratio
    top_ctr['ctr'] = round(top_ctr['click']  / top_ctr['impression'],2)

    # Casteamos los valores infinitos como un ctr del 100%, asumiento un typo en el registro de la impresión
    top_ctr['ctr'] = np.where(np.isinf(top_ctr['ctr']),1,top_ctr['ctr'])

    # Ordenamos por los valores más altos
    top_ctr = top_ctr.sort_values(by = ['advertiser_id', 'ctr'], ascending = [True, False])

    # Obtenemos los 20 productos con mayor conversión por advertiser
    resultado_top_ctr = top_ctr.groupby('advertiser_id').head(20)

    # Nos quedamos con las columnas de interés
    resultado_top_ctr['log'] = dias_unicos[i]
    resultado_top_ctr = resultado_top_ctr[['log','advertiser_id','product_id']]

    # Appendeamos
    resultados_totales_top_ctr = pd.concat([resultados_totales_top_ctr, resultado_top_ctr], ignore_index=True)

    ############################
    # Calculamos el TopProduct #
    ############################    

    # Hacemos las agrupaciones
    datos_top_pro = datos_top_pro.groupby(['advertiser_id', 'product_id']).count().reset_index()
    
    # Ordenamos
    top_product = datos_top_pro.sort_values(by = ['advertiser_id', 'date'], ascending = [True, False])    

    # Nos quedamos con los 20 productos más vistos por advertiser
    resultados_top_product = top_product.groupby('advertiser_id').head(20)

    # Nos quedamos con las columnas de interés
    resultados_top_product['log'] = dias_unicos[i]
    resultados_top_product = resultados_top_product[['log','advertiser_id','product_id']]

    # Appendeamos
    resultados_totales_top_product = pd.concat([resultados_totales_top_product, resultados_top_product], ignore_index=True)

<timed exec>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

CPU times: user 1.44 s, sys: 0 ns, total: 1.44 s
Wall time: 1.44 s


<timed exec>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [7]:
display(resultados_totales_top_product.shape)
display(resultados_totales_top_ctr.shape)

(23600, 3)

(23600, 3)

In [8]:
# Guardamos los resultados
#resultados_totales_top_ctr.to_csv('/home/pepino/Desktop/Maestría Udesa/Segundo Año/Programación Avanzada para grandes volúmenes de datos/TP/resultados_totales_top_ctr.csv', index = False, header = True)

#resultados_totales_top_product.to_csv('/home/pepino/Desktop/Maestría Udesa/Segundo Año/Programación Avanzada para grandes volúmenes de datos/TP/resultados_totales_top_product.csv', index = False)


In [ ]:
fechas_ordenadas = sorted(list(resultados_totales_top_product[resultados_totales_top_product['advertiser_id'] == '03KNVBO915KY2ZPGA57J']['log'].unique()))

In [209]:
fechas_ordenadas = sorted(list(resultados_totales_top_product[resultados_totales_top_product['advertiser_id'] == '03KNVBO915KY2ZPGA57J']['log'].unique()), reverse = True)[0:7]
print(fechas_ordenadas)
#resultados_totales_top_product[resultados_totales_top_product['advertiser_id'] == '03KNVBO915KY2ZPGA57J'].sort_values(by = ['log'], ascending = [False]).head()

['2023-05-30', '2023-05-29', '2023-05-28', '2023-05-27', '2023-05-26', '2023-05-25', '2023-05-24']


# Ya tenemos la base completa con toda la historia

In [9]:
resultados_totales_top_ctr.head()

,log,advertiser_id,product_id
0,2023-04-02,03KNVBO915KY2ZPGA57J,0zmyxz
1,2023-04-02,03KNVBO915KY2ZPGA57J,16n8be
2,2023-04-02,03KNVBO915KY2ZPGA57J,28ycmv
3,2023-04-02,03KNVBO915KY2ZPGA57J,3calwd
4,2023-04-02,03KNVBO915KY2ZPGA57J,3mizxh


In [10]:
# Levantamos los resultados de cada modelo
# TopCTR
resultados_top_ctr = pd.read_csv('/home/pepino/Desktop/Maestría Udesa/Segundo Año/Programación Avanzada para grandes volúmenes de datos/TP/resultados_totales_top_ctr.csv', header = 0)
resultados_top_ctr = resultados_top_ctr[(resultados_top_ctr['advertiser_id'] == 'SOVPFK3BBWKTQM9HOHWJ')]

# TopProduct
resultados_top_product = pd.read_csv('/home/pepino/Desktop/Maestría Udesa/Segundo Año/Programación Avanzada para grandes volúmenes de datos/TP/resultados_totales_top_product.csv', header = 0)
resultados_top_product = resultados_top_product[(resultados_top_product['advertiser_id'] == 'SOVPFK3BBWKTQM9HOHWJ')]

# Compactamos por el advertiser que se requiere
resultados_adv = pd.concat([resultados_top_ctr, resultados_top_product], ignore_index=True)

In [11]:
resultados_adv.head()

,log,advertiser_id,product_id
0,2023-04-02,SOVPFK3BBWKTQM9HOHWJ,zfozhl
1,2023-04-02,SOVPFK3BBWKTQM9HOHWJ,0b5rmh
2,2023-04-02,SOVPFK3BBWKTQM9HOHWJ,0zydzi
3,2023-04-02,SOVPFK3BBWKTQM9HOHWJ,15c15q
4,2023-04-02,SOVPFK3BBWKTQM9HOHWJ,15pp6q


In [12]:
# Agarramos los últimos 7 días
fechas_ordenadas = sorted(list(resultados_adv['log'].unique()), reverse = True)[0:7]

In [13]:
fechas_ordenadas

['2023-05-30',
 '2023-05-29',
 '2023-05-28',
 '2023-05-27',
 '2023-05-26',
 '2023-05-25',
 '2023-05-24']

In [14]:
# Filtramos los productos recomendados para el advertiser en esos días
filtrado = resultados_adv[resultados_adv['log'].isin(fechas_ordenadas)]['product_id'].unique()

In [15]:
[filtrado]

[array(['03j6wc', '0wfrps', '10axas', '15c15q', '15pp6q', '1bjt9f',
        '2nqzn1', '3b1glx', '4ej8w6', '4f55gk', '6peupa', '73k7gz',
        '7wubsu', '87lp1s', '8asss7', '8e6tva', 'acbohu', 'ajuebw',
        'c1h2vn', 'c2n98t', '0b5rmh', '0zydzi', '2dhydo', '7qub1p',
        '8kc2gc', 'e5tejo', 'f00j3c', 'hrhmnp', 'i4756r', 'icclta',
        'igzucd', 'iii41j', 'j7ua0b', 'x4c2jl', '0q0kg3', '2q9qmd',
        '3pulgl', '8dxuq2', 'bdsobl', '2xstvh', '6fa8lw', '9rmu1i',
        'q6g402', 'dxib3c', 'iwi7hu', 'jux83g', 'g3ivrt', 'gvlbio',
        'c6nfma', 'j031km', 'keol8b', 'ltpkiv', 'm76e67', 'mg3tnw',
        'ilu1sc', 'qks5un', 'dkmr5q', 'ofym43', 'ov1lyn', 'pd0lqx',
        'rd3pgw', 'ut5007', 'v24hu1', 'q15do6', 'td3xe5', 'y9gu5t',
        'wmnc0m', 'tdzpxc', 'gv9o9j', 'nwzsc9', 'oocfg0', 'vedvan',
        'npdpya', 'xxc3zn', 'zfozhl', 'mffo1x', 'rpbx90', 'slrequ',
        'odw4nw', 'sc0smc', 'wd0rzt', 'kanmlg', 'qpzv5r', 'vvt39m',
        'l46sui', 'tgyf69', 'h3qtxn', 'ly38u0', 

In [16]:
# Chequeo manual
prueba1 = resultados_totales_top_product[(resultados_totales_top_product['log'] == '2023-04-06') & 
                                         (resultados_totales_top_product['advertiser_id'] == '03KNVBO915KY2ZPGA57J')]['product_id'].unique()
prueba1

array(['2b9w5m', 'nns2gr', '16n8be', '2kr3c3', '3calwd', 'dbljzg',
       'hrubu3', 'kj03qz', 'n6sqm9', 'owrdhk', 'yjzee5', 'z4pnuj',
       '1f2p9m', '28ycmv', '40jw2m', '4hzw8j', '4mudad', '59zz7y',
       '5vpluf', '76kb18'], dtype=object)

In [17]:
# Chequeo manual
prueba2 = resultados_totales_top_ctr[(resultados_totales_top_ctr['log'] == '2023-04-06') & 
                                     (resultados_totales_top_ctr['advertiser_id'] == '03KNVBO915KY2ZPGA57J')]['product_id'].unique()
prueba2

array(['q21r4u', '0zmyxz', '16n8be', '1lsjl2', '28ycmv', '2b9w5m',
       '3calwd', '3mizxh', '40jw2m', '4mudad', '4xbwqh', '59zz7y',
       '5vpluf', '76kb18', '94b5uf', '94sb3v', 'ar4i1s', 'csg8br',
       'dbljzg', 'djcjem'], dtype=object)

In [18]:
cruce = [value for value in prueba1 if value in prueba2]
len(cruce)

10

In [19]:
# Unimos, pero me va a generar 20 rows de más por cada producto
union = resultados_totales_top_ctr.merge(resultados_totales_top_product, on = ['log', 'advertiser_id'], suffixes=['_top_ctr', '_top_product'])

# Hago un agrupamiento por advertiser y cuento el número de coincidencias
conteo_coincidencias = union.groupby(['advertiser_id','log']).apply(lambda x: (x['product_id_top_ctr'] == x['product_id_top_product']).sum())

# Calculo la coincidencia por cada advertiser: (coincidencias / coincidencias_posibles) %
porcentaje_coincidencia = conteo_coincidencias / 20 * 100

# Armamos unas base con las coincidencias
porcentajes_adv_log = porcentaje_coincidencia.reset_index()
porcentajes_adv_log.rename(columns = {0:'percen'}, inplace = True)

# Hacemos la coincidencia promedio por advertiser
porcen_agroup = porcentajes_adv_log.groupby('advertiser_id')['percen'].mean().reset_index()

# Agarramos los 20 advertisers con mayor porcentaje de coincidencia promedio entre ambos modelos
adv_con_mayor_coincidencia = porcen_agroup.reset_index().sort_values(by = 'percen', ascending = False).reset_index()[['advertiser_id','percen']].head(20)

In [20]:
np.round(porcen_agroup['percen'].mean(),2)

27.86

In [21]:
porcentajes_adv_log.groupby('advertiser_id')['percen'].std().reset_index()

,advertiser_id,percen
0,03KNVBO915KY2ZPGA57J,9.630824
1,1OW4LWJSBJK6E1D9FDH7,9.727437
2,2WPF1NXECF3G6NUMWDO7,10.099592
3,5E325T5HYL61QSABVR5V,10.840618
4,6X20RDH567MX2X3TXYJ7,9.204953
5,8C88YB6E8YCGWU07HA7A,7.927381
6,9Z77N44VDW6KX6VBWJ4X,9.694335
7,AK81O7W3KGPEN8LABG2N,8.648436
8,EN1SA43DTN2LIR8DEW5S,10.748574
9,GXLKAA83238FVRU718EX,8.956352


In [22]:
#porcen_agroup.sort_values(by = '').head(20)
porcen_agroup.reset_index().sort_values(by = 'percen', ascending = False).reset_index()[['advertiser_id','percen']].head(20)

,advertiser_id,percen
0,IOBPI63RBJIHI5FB7U9O,30.677966
1,LW045DVYSGRD75TK6U54,30.338983
2,6X20RDH567MX2X3TXYJ7,29.576271
3,IDOFCO721HTJGDH7332G,29.067797
4,03KNVBO915KY2ZPGA57J,28.728814
5,L6WM4665XZVS9MCZRTVI,28.474576
6,SOVPFK3BBWKTQM9HOHWJ,28.474576
7,M0LU6DCI1WILGQBZ6808,28.305085
8,8C88YB6E8YCGWU07HA7A,28.135593
9,5E325T5HYL61QSABVR5V,27.711864


In [23]:
union[union['advertiser_id'] == '03KNVBO915KY2ZPGA57J']

,log,advertiser_id,product_id_top_ctr,product_id_top_product
0,2023-04-02,03KNVBO915KY2ZPGA57J,0zmyxz,7rbqhz
1,2023-04-02,03KNVBO915KY2ZPGA57J,0zmyxz,csg8br
2,2023-04-02,03KNVBO915KY2ZPGA57J,0zmyxz,4nbvel
3,2023-04-02,03KNVBO915KY2ZPGA57J,0zmyxz,4xbwqh
4,2023-04-02,03KNVBO915KY2ZPGA57J,0zmyxz,94b5uf
...,...,...,...,...
464395,2023-05-30,03KNVBO915KY2ZPGA57J,f6h2nn,4mudad
464396,2023-05-30,03KNVBO915KY2ZPGA57J,f6h2nn,4nbvel
464397,2023-05-30,03KNVBO915KY2ZPGA57J,f6h2nn,4xbwqh
464398,2023-05-30,03KNVBO915KY2ZPGA57J,f6h2nn,59zz7y


In [24]:
productos_top_ctr = set(resultados_totales_top_ctr['product_id'].unique())

In [25]:
productos_top_product = set(resultados_totales_top_product['product_id'].unique())

In [26]:
productos_top_ctr - productos_top_product

set()

In [27]:
len(list(productos_top_product - productos_top_ctr))

611

In [28]:
productos_top_ctr - set(list(productos_top_ctr)[1:])

{'18v6fo'}

In [29]:
recomendaciones_totales = pd.concat([resultados_totales_top_ctr, resultados_totales_top_product], ignore_index=True)

In [30]:
recomendaciones_totales.groupby('product_id')['product_id'].value_counts().reset_index().sort_values(by = 'count', ascending = False).head(10)

,product_id,count
13,073bf1,66
182,39d2xv,66
160,2ucg8e,66
54,0zmyxz,66
59,10axas,65
62,14829e,64
162,2uqhzn,64
2,01mgr3,64
156,2ppn5x,63
11,05p8w3,63


In [31]:
prueba = {}

In [32]:
prueba['recomendaciones_prueba'] = recomendaciones_totales.groupby('product_id')['product_id'].value_counts().reset_index().sort_values(by = 'count', ascending = False).head(10).reset_index()[['product_id','count']]

In [33]:
print(prueba)

{'recomendaciones_prueba':   product_id  count
0     073bf1     66
1     39d2xv     66
2     2ucg8e     66
3     0zmyxz     66
4     10axas     65
5     14829e     64
6     2uqhzn     64
7     01mgr3     64
8     2ppn5x     63
9     05p8w3     63}


In [34]:
import pandas as pd

# Create example DataFrame
df = pd.DataFrame({'date': ['2023-04-12', '2023-04-13', '2023-04-14', '2023-04-15', '2023-04-16',
                            '2023-04-17', '2023-04-18', '2023-04-19', '2023-04-20', '2023-04-21'],
                   'product': ['A', 'A', 'B', 'B', 'B', 'A', 'A', 'A', 'B', 'B']})

# Group by product and create a new column with the length of each sequential run of product names
df['sequential_runs'] = df.groupby('product')['product'].apply(lambda x: x.ne(x.shift()).cumsum().value_counts())

# Get the index of the row with the longest sequential run for each product
max_run_indices = df.groupby('product')['sequential_runs'].apply(lambda x: x.idxmax())

# Get the product name for each max_run_index
max_products = df.loc[max_run_indices, 'product']

# Print the results
print(max_products)

TypeError: incompatible index of inserted column with frame index

In [36]:
resultados_totales_top_ctr.head()

,log,advertiser_id,product_id
0,2023-04-02,03KNVBO915KY2ZPGA57J,0zmyxz
1,2023-04-02,03KNVBO915KY2ZPGA57J,16n8be
2,2023-04-02,03KNVBO915KY2ZPGA57J,28ycmv
3,2023-04-02,03KNVBO915KY2ZPGA57J,3calwd
4,2023-04-02,03KNVBO915KY2ZPGA57J,3mizxh


In [38]:
import pandas as pd

# create a sample dataframe
df = pd.DataFrame({
    'product_id': [1, 1, 2, 2, 3, 3, 1, 1, 1, 4, 4],
    'date': ['2022-01-01', '2022-01-02', '2022-01-02', '2022-01-03',
             '2022-01-03', '2022-01-04', '2022-01-04', '2022-01-05',
             '2022-01-06', '2022-01-06', '2022-01-07']
})

# convert the date column to datetime
df['date'] = pd.to_datetime(df['date'])

# sort the dataframe by date
df = df.sort_values('date')

# assign sequential group numbers to each product
df['group'] = df['product_id'].ne(df['product_id'].shift()).cumsum()

# calculate the duration of each product's presence in each group
group_durations = df.groupby(['product_id', 'group'])['date'].apply(lambda x: x.diff().sum())

# sum the durations for each product across all groups
total_durations = group_durations.groupby('product_id').sum()

# find the product(s) with the highest total duration
longest_duration_products = total_durations[total_durations == total_durations.max()].index.tolist()

print('Product(s) with longest sequential duration:', longest_duration_products)


Product(s) with longest sequential duration: [1]


In [ ]:
import pandas as pd

# load your dataframe
df = pd.read_csv('your_data.csv')

# sort the dataframe by date in ascending order
df = df.sort_values('date')

# group the dataframe by product ID
groups = df.groupby('product_id')

# calculate the difference between consecutive dates for each group
df['diff'] = groups['date'].diff()

# calculate the cumulative sum of timedelta values
df['cumulative_diff'] = df.groupby('product_id')['diff'].cumsum()

# sort the dataframe in descending order by the cumulative difference column
result = df.sort_values('cumulative_diff', ascending=False)

# get the list of products that have been present sequentially for the longest period of time
longest_seq_products = result['product_id'].unique()[:10] # top 10 products

In [128]:
# Nos quedamos con lo que son las fechas y el producto, eliminando duplicados en la base
base_productos_dia = resultados_totales_top_ctr[['log','product_id']].drop_duplicates()
base_productos_dia['date'] = pd.to_datetime(base_productos_dia['log']) # Lo casteamos a datetime para que podamos hacer lo que serían los delta de fechas

# Acomodamos para poder sacar las diferencias 
base_productos_dia = base_productos_dia.sort_values(by = 'date')
groups = base_productos_dia.groupby('product_id')
base_productos_dia['diff'] = groups['date'].diff()

# Casteamos a númerico para que después sea más fácil hacer las operaciones
base_productos_dia['diff'] = base_productos_dia['diff'].astype('timedelta64[ns]').dt.days

# Obtenemos lo que es el producto que tuvo mayor cantidad de días consecutivos como producto recomendado
base_productos_dia[base_productos_dia['diff'] == 1].groupby('product_id').agg({'diff':'count'}).idxmax()

In [129]:
base_productos_dia.tail()

,log,product_id,date,diff
23327,2023-05-30,2buw8n,2023-05-30,2.0
23326,2023-05-30,25c4hl,2023-05-30,6.0
23325,2023-05-30,1jv8ck,2023-05-30,2.0
23335,2023-05-30,7yl2eo,2023-05-30,2.0
23599,2023-05-30,mg3tnw,2023-05-30,NaN


In [147]:
base_productos_dia[base_productos_dia['diff'] == 1].groupby('product_id').agg({'diff':'count'}).idxmax()

diff    9mf0cn
dtype: object

In [109]:

base_productos_dia['diff'].astype('timedelta64[ns]').dt.days.tail()


23327    2.0
23326    6.0
23325    2.0
23335    2.0
23599    NaN
Name: diff, dtype: float64

In [101]:
base_productos_dia[base_productos_dia['product_id'] == 'mg3tnw']

,log,product_id,date,diff
23599,2023-05-30,mg3tnw,2023-05-30,NaT


In [94]:
base_productos_dia[base_productos_dia['product_id'] == 'bei3gv']

,log,product_id,date,diff,cumulative_diff
133,2023-04-02,bei3gv,2023-04-02,NaT,NaT
2138,2023-04-07,bei3gv,2023-04-07,5 days,5 days
2538,2023-04-08,bei3gv,2023-04-08,1 days,6 days
2937,2023-04-09,bei3gv,2023-04-09,1 days,7 days
4120,2023-04-12,bei3gv,2023-04-12,3 days,10 days
4936,2023-04-14,bei3gv,2023-04-14,2 days,12 days
6132,2023-04-17,bei3gv,2023-04-17,3 days,15 days
6931,2023-04-19,bei3gv,2023-04-19,2 days,17 days
7336,2023-04-20,bei3gv,2023-04-20,1 days,18 days
7733,2023-04-21,bei3gv,2023-04-21,1 days,19 days


In [95]:
df = pd.DataFrame({
    'date': ['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-05', '2022-01-06', '2022-01-07', '2022-01-08'],
    'product_id': [1, 2, 3, 2, 3, 4, 1]
})

In [96]:
df

,date,product_id
0,2022-01-01,1
1,2022-01-02,2
2,2022-01-03,3
3,2022-01-05,2
4,2022-01-06,3
5,2022-01-07,4
6,2022-01-08,1
